# Regression Random Forest Lösung

### Module importieren

In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

### Daten einlesen

In [2]:
# adjust to correct path if necessary
dataset_dir_path = os.path.join(os.path.pardir, os.path.pardir, os.path.pardir, os.path.pardir, 'datasets', 'regression')
rental_bikes_df = pd.read_csv(os.path.join(dataset_dir_path, 'rental_bikes.csv'))

### Überblick über Daten bekommen

In [3]:
rental_bikes_df.head()

,Date,Rented Bike Count,Hour,Temperature,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature,Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [4]:
rental_bikes_df.describe().round(2)

,Rented Bike Count,Hour,Temperature,Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature,Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
count,8760.00,8760.00,8760.00,8760.00,8760.00,8760.00,8760.00,8760.00,8760.00,8760.00
mean,704.60,11.50,12.88,58.23,1.72,1436.83,4.07,0.57,0.15,0.08
std,645.00,6.92,11.94,20.36,1.04,608.30,13.06,0.87,1.13,0.44
min,0.00,0.00,-17.80,0.00,0.00,27.00,-30.60,0.00,0.00,0.00
25%,191.00,5.75,3.50,42.00,0.90,940.00,-4.70,0.00,0.00,0.00
50%,504.50,11.50,13.70,57.00,1.50,1698.00,5.10,0.01,0.00,0.00
75%,1065.25,17.25,22.50,74.00,2.30,2000.00,14.80,0.93,0.00,0.00
max,3556.00,23.00,39.40,98.00,7.40,2000.00,27.20,3.52,35.00,8.80


### Daten vorbereiten

#### Ziel Variable als numpy Array in `y` speichern

In [5]:
y = rental_bikes_df['Rented Bike Count'].values

In [6]:
# rental_bikes_df['weekday'] = pd.to_datetime(rental_bikes_df.Date, dayfirst=True ).dt.weekday

#### Zielvariable und nicht benötigte Variable entfernen

In [7]:
rental_bikes_df_features = rental_bikes_df.drop(columns=['Rented Bike Count', 'Date'])

#### Kategorische Variablen in Dummyvariablen umwandeln

In [8]:
rental_bikes_df_features = pd.get_dummies(rental_bikes_df_features, columns=['Seasons', 'Holiday', 'Functioning Day'], drop_first=True)
X = rental_bikes_df_features.values

#### Für spätere Validierung Daten in Train- und Testset aufteilen

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=5)

### Modell Training

Initialisieren Sie das `RandomForestRegressor` Modell und wählen Sie dabei Parameterwerte aus. In der Dokumentation von `RandomForestRegressor` können Sie mögliche Parameter mit Erklärung finden. Nutzen Sie die `help()` Funktion um Sie aufzurufen. Mögliche Parameter wären z.B.: `n_estimators`, `max_depth`, `min_samples_split / min_samples_leaf`, `max_features`.

Anschließend nutzen Sie `fit()` zum Trainieren des Modells. Sie können auch verschiedene Kombinationen von Parameterwerten ausprobieren um zu sehen wie es sich auf das Ergebnis oder die Trainingszeit auswirkt. 

In [10]:
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble._forest:

class RandomForestRegressor(ForestRegressor)
 |  RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators 

In [11]:
rf_model = RandomForestRegressor(n_estimators=200, min_samples_split=10)

In [12]:
rf_model.fit(X_train, y_train)

RandomForestRegressor(min_samples_split=10, n_estimators=200)

### Modell anwenden

Nachdem Sie das Modell trainiert haben können Sie es nutzen um Vorhersagen für das Testset zu treffen. Nutzen Sie dafür `predict()` und speichern Sie die Vorhersagen in `y_pred`.

In [13]:
y_pred = rf_model.predict(X_test)

### Validierung

Überprüfen Sie im letzten Schritt, wie gut Ihr Modell Vorhersagen machen kann anhand des RMSE (Root Mean Squared Error) und vergleichen Sie es damit den Durchschnitt vorherzusagen. 

In [14]:
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred)).round()
print(rmse_test)

240.0


In [15]:
y_mean = np.repeat(np.mean(y_train), len(y_test))

rmse_baseline = np.sqrt(mean_squared_error(y_test, y_mean)).round()
print(rmse_baseline)

646.0
